In [1]:
import torch

from packages import *

from BranchAndBound import BranchAndBound
from NeuralNetwork import NeuralNetwork

In [2]:
# fileName = "Test3-5-3.pth"
# pathToStateDictionary = "Networks/" + fileName
# network = NeuralNetwork(pathToStateDictionary)
# dim = network.Linear[0].weight.shape[1]

# outputDim = network.Linear[-1].weight.shape[0]
# device = torch.device("cpu")
# network.to(device)

# W = []
# bias = []
# print(network.Linear)
# for i in range(len(network.Linear)):
#     if i%2:
#         continue
#     W.append(torch.Tensor(network.Linear[i].weight))
#     bias.append(torch.Tensor(network.Linear[i].bias))


# W, bias

In [3]:
# A = torch.zeros((2*dim, 2*dim))
# A[:dim, dim:] = torch.eye(dim)
# A

In [4]:
# g = 9.8

# B = torch.zeros((2*dim, dim))
# B[dim:] = torch.eye(dim)
# B[3,0] = g
# B[4,1] = -g
# B

In [5]:
# class NeuralNetworkforReachSDP(nn.Module):
#     def __init__(self, path, A, B):
#         self.statedim = 6
#         self.controlstate = 3

#         super().__init__()
#         stateDictionary = torch.load(path, map_location=torch.device("cpu"))
#         layers = []
#         for keyEntry in stateDictionary:
#             if "weight" in keyEntry:
#                 layers.append(nn.Linear(stateDictionary[keyEntry].shape[1] + self.statedim,
#                                              stateDictionary[keyEntry].shape[0] + self.statedim))
#                 layers.append(nn.ReLU())
        
#         layers.append(nn.Linear(self.statedim + self.controlstate, self.statedim))
        
#         self.Linear = nn.Sequential(
#             *layers
#         )

#         print(self.Linear[0].weight)
#         for i in range(len(network.Linear)):
#             if i%2:
#                 continue
#             self.Linear[i].weight[]
        
#         # print(self.state_dict)
#         # print(self.Linear[0].weight)

#     def load(self, path):
#         stateDict = torch.load(path, map_location=torch.device("cpu"))
#         self.load_state_dict(stateDict)

#     # @TODO
#     # def train(self):

#     def forward(self, x):
#         return self.Linear(x)

In [6]:
# network = NeuralNetworkforReachSDP(pathToStateDictionary, A, B)

### Approach 2

In [28]:
import scipy.io

In [37]:
def loadData(fileLocation="./Networks/nnmpc_nets_di_1.mat"):
    nn = scipy.io.loadmat(fileLocation)
    if 'net' in nn.keys():
        net = nn['net']
        bs = [net['biases'][0, 0][0, 0], net['biases'][0, 0][0, 1], net['biases'][0, 0][0, 2]]
        Ws = [net['weights'][0, 0][0, 0], net['weights'][0, 0][0, 1], net['weights'][0, 0][0, 2]]
        A = nn['AMatrix']
        B = nn['BMatrix']
        lu = 10 * (np.random.rand(2, 2) - 0.5)
    else:
        bs = [nn['biases'][0, 0], nn['biases'][0, 1], nn['biases'][0, 2]]
        Ws = [nn['weights'][0, 0], nn['weights'][0, 1], nn['weights'][0, 2]]
        
        A = np.array([[1, 1], [0, 1]])
        B = np.array([[0.5], [1]])

        lu = np.array([[1, 2],
                   [1.5, 2.5]])

    uDimension = Ws[-1].shape[0]
    # uDimension = 1  # the code is not designed for other scenarios
    uBounds = np.array([[-np.pi/9, np.pi/9],
                        [-np.pi/9, np.pi/9],
                        [0, 2*9.8]])

    if uBounds.shape[0] != uDimension:
        raise "Error in uBounds"

    Ws.append(-np.eye(uDimension))
    Ws.append(-np.eye(uDimension))
    bs[-1] -= uBounds[:, 0:1]
    bs.append(uBounds[:, 1:2] - uBounds[:, 0:1])
    bs.append(uBounds[:, 1:2])

    dimensionList = [w.shape for w in Ws]
    return A, B, Ws, bs, lu, dimensionList


In [52]:
A, B, Ws, bs, lu, dimensionList = loadData('./Networks/quadRotorv2.0.mat')
print(dimensionList)

[(32, 6), (32, 32), (3, 32), (3, 3), (3, 3)]


In [53]:
class NeuralNetworktemp(nn.Module):
    def __init__(self, Ws, bs, dimensionList):
        super().__init__()
        layers = []
        for i in range(len(dimensionList)):
            netSize = dimensionList[i]

            layers.append(nn.Linear(netSize[1], netSize[0]))
            layers[-1].weight = torch.nn.parameter.Parameter(torch.from_numpy(Ws[i]).float())
            layers[-1].bias = torch.nn.parameter.Parameter(torch.from_numpy(bs[i][:, 0]).float())
            layers.append(nn.ReLU())

        layers.pop()
        self.Linear = nn.Sequential(
            *layers
        )

        print(self)

    def forward(self, x):
        return self.Linear(x)


In [54]:
nnet = NeuralNetworktemp(Ws, bs, dimensionList)
torch.save(nnet.state_dict(), './Networks/quadRotorv2.0.pth')

NeuralNetworktemp(
  (Linear): Sequential(
    (0): Linear(in_features=6, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=3, bias=True)
    (5): ReLU()
    (6): Linear(in_features=3, out_features=3, bias=True)
    (7): ReLU()
    (8): Linear(in_features=3, out_features=3, bias=True)
  )
)


In [44]:
print(nnet.Linear[0].bias)

Parameter containing:
tensor([-0.0366, -0.5006, -0.0287,  0.2468,  0.1089, -0.1874, -0.0308,  0.0423,
         0.1505, -0.3510, -0.4049, -0.1228, -0.0959, -0.0497, -0.2564, -0.0347,
        -0.3925, -0.1830, -0.0517, -0.2847, -0.0009, -0.3425, -0.0375, -0.2366,
        -0.1291, -0.0923, -0.2565,  0.2335,  0.0200, -0.2238,  0.2743,  0.1077],
       requires_grad=True)


### AProach 3

In [8]:
from packages import *


class NeuralNetwork(nn.Module):
    def __init__(self, path, A=None, B=None, c=None):
        super().__init__()
        stateDictionary = torch.load(path, map_location=torch.device("cpu"))['stateDict']
        layers = []
        for keyEntry in stateDictionary:
            if "weight" in keyEntry:
                layers.append(nn.Linear(stateDictionary[keyEntry].shape[1], stateDictionary[keyEntry].shape[0]))
                layers.append(nn.ReLU())
        layers.pop()
        self.Linear = nn.Sequential(
            *layers
        )
        self.rotation = nn.Identity()
        print(self)
        print(stateDictionary)
        dictionary = {}
        for i in [0, 2, 4]:
            temp_w = "Linear." + str(i) +  ".weight"
            temp_b = "Linear." + str(i) +  ".bias"
            dictionary[temp_w] = stateDictionary[str(i) + '.weight']
            dictionary[temp_b] = stateDictionary[str(i) + '.bias']

        self.load_state_dict(dictionary)
        
        self.A = A
        self.B = B
        self.c = c
        if self.A is None:
            dimInp = self.Linear[0].weight.shape[1]
            self.A = torch.zeros((dimInp, dimInp)).float()
            self.B = torch.eye((dimInp)).float()
            self.c = torch.zeros(dimInp).float()
        self.repetition = 1

    def load(self, path):
        stateDict = torch.load(path, map_location=torch.device("cpu"))
        self.load_state_dict(stateDict)

    def forward(self, x):
        x = self.rotation(x)
        for i in range(self.repetition):
            x = x @ self.A.T + self.Linear(x) @ self.B.T + self.c
        return x

In [9]:
fileLocation="./Networks/doubleIntegrator_reachlp.pth"
nn = NeuralNetwork(fileLocation)

NeuralNetwork(
  (Linear): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
  (rotation): Identity()
)
OrderedDict([('0.weight', tensor([[ 0.0096,  0.0736],
        [ 0.3017, -0.6268],
        [-0.4645,  0.5666],
        [ 0.6853,  0.7431],
        [ 0.1216, -0.5499],
        [-0.4705,  0.6522],
        [ 0.5109,  0.5491],
        [ 0.6694,  0.5562],
        [-0.1820, -0.3704],
        [-0.1960,  0.0544]])), ('0.bias', tensor([-0.0483,  0.0941,  0.0000,  0.0381,  0.2122,  0.0000,  0.0650,  0.0197,
        -0.1224,  0.0000])), ('2.weight', tensor([[ 0.5235,  0.1980,  0.2334,  0.3425, -0.6947,  0.5360,  0.2231, -0.4651,
          0.1069,  0.1825],
        [-0.0959, -0.1040, -0.5321,  0.5866,  0.2049, -0.4737,  0.4564,  0.5498,
          0.4233, -0.3574],
        [ 0.4046,  0.4450, -0.2218,  0.2496,  0.4490,  0.1242,

In [10]:
torch.save(nn.state_dict(), './Networks/doubleIntegrator_reachlp.pth')
